<a href="https://colab.research.google.com/github/aknip/Langchain-etc./blob/main/Langchain-Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain Agent

1. Run "Setup..."
2. Run one of the Agent sections
3. Run Gradio app or CLI apps


# Setup for all agents and apps

In [ ]:
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])

In [ ]:
!pip install langchain langchainhub langchain_experimental openai gradio -q
!pip install google-search-results wikipedia -q
%load_ext gradio

In [ ]:
import json
import os
from getpass import getpass
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

In [6]:
from langchain.agents import Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper, SQLDatabase, WikipediaAPIWrapper
from langchain_experimental.sql import SQLDatabaseChain
from langchain.tools import WikipediaQueryRun
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool, StructuredTool
import os
import langchain
import openai
import textwrap
import gradio as gr
import random
import time

# Agent 1: Wikpedia, Search, Music DB

In [ ]:
# Download "Chinook" Music Sales DB
import os
import requests
import zipfile
fname = 'chinook.zip'
url = 'https://www.sqlitetutorial.net/wp-content/uploads/2018/03/' + fname
r = requests.get(url)
open(fname, 'wb').write(r.content)
zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

In [ ]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential']
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')
search = SerpAPIWrapper(serpapi_api_key=CREDS['SERP-API']['key']['credential'])
db = SQLDatabase.from_uri("sqlite:///chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about real-time events. You should ask targeted questions"
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to answer questions about a big picture or background of something."
    ),
    Tool(
        name="MusicSales",
        func=db_chain.run,
        description="useful for when you need to answer questions about mucis sales in a store. Should be strickly follow the tables info."
    )
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()




# Agent 2: Calculate insurance premium

In [18]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential']
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')

def calculate_insurance_premium(insured_sum: int, industry: str) -> float:
    """Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer."""
    premium = insured_sum*0.25
    return premium

tools = [
    StructuredTool.from_function(calculate_insurance_premium, return_direct=False)
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()


# Gradio Chat App for Agent

In [16]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

#
# Gradio app
#

# Theming
theme = gr.themes.Default(
    primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
)
# Styling: Change max width
css = """
  .gradio-container {max-width: 800px!important}
  .vspacer1 {margin-top: 20px}
"""

with gr.Blocks(theme=theme, css=css) as demo:

    gr.Markdown("# Agent Chat", elem_classes="vspacer1")

    with gr.Tab("Chat"):
      # https://www.gradio.app/docs/chatbot

      chatbot = gr.Chatbot(bubble_full_width=False)
      msg = gr.Textbox()
      clear = gr.ClearButton([msg, chatbot])

      # Init agent and memory
      memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
      agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
      messages = [{"role": "assistant", "content": "How can I help you?"}]
      #chatbot.append(None, "How can I help you?")

      def ask(message, chat_history):
          chat_history.append((message, None))
          messages.append({"role": "user", "content": message})
          return "", chat_history

      def respond(chat_history):
          prompt = chat_history[-1][0] # get prompt from history (last entry)
          response = agent_executor.invoke({"input": prompt})
          msg = {"role": "assistant", "content": response["output"]}
          messages.append(msg)
          chat_history.append((None, response["output"]))
          print("\n\nMemory from response object:")
          print(textwrap.fill(str(response["chat_history"]), 80))
          return chat_history

      msg.submit(ask, [msg, chatbot], [msg, chatbot], queue=False).then(
        respond, chatbot, chatbot
      )

demo.launch(quiet=True, share=False, debug=True)

<IPython.core.display.Javascript object>



> Entering new AgentExecutor chain...
Sure, I can help with that. However, I also need to know the industry of the customer to calculate the insurance premium. Could you please provide that information?

> Finished chain.


Memory from response object:
[HumanMessage(content='Give me the premium for an insured sum of 10000000'),
AIMessage(content='Sure, I can help with that. However, I also need to know the
industry of the customer to calculate the insurance premium. Could you please
provide that information?')]


> Entering new AgentExecutor chain...

Invoking: `calculate_insurance_premium` with `{'insured_sum': 10000000, 'industry': 'metal construction'}`


2500000.0The premium for an insured sum of 10,000,000 in the metal construction industry is 2,500,000.

> Finished chain.


Memory from response object:
[HumanMessage(content='Give me the premium for an insured sum of 10000000'),
AIMessage(content='Sure, I can help with that. However, I also need to know the
industry of the custo

# CLI für Agent (Single Prompt)

In [20]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False,  memory=memory)

langchain.debug = True

prompt = "Give me the premium for an insured sum of 10000000 for metal construcion industry"
# prompt = "Wie viele Menschen leben in den USA?"
# prompt = "How many people live in the US?"
# prompt = "What is the best selling song in the music store?"
# prompt = "What is the cheapest album in the music store? Tell me the price."
# prompt = "What is the best selling song starting with the letter 'F' in the music store? Tell me title and artist."
# prompt = "How is the weather now in Cologne? Please use Celsius as unit for temperature."
response = agent_executor.invoke({"input": prompt})

# Response-Objekte:
print()
print(textwrap.fill(str(response["input"]), 80))
print()
print(textwrap.fill(str(response["output"]), 80))
print()
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
for msg in msg_history:
  print(msg)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Give me the premium for an insured sum of 10000000 for metal construcion industry",
  "chat_history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Give me the premium for an insured sum of 10000000 for metal construcion industry",
  "chat_history": [],
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableMap] Entering Chain run with input:
{
  "input": "Give me the premium for an insured sum of 10000000 for metal construcion industry",
  "chat_history": [],
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableMap > 4:chain:<lambda>] Entering Chain run with input:
{
  "input": "Give me the premium for an insured sum of 10000000 for metal construcion industry",
  "chat_history": [],
  "intermediate_steps": []
}
[chain/end] 

In [ ]:

print(textwrap.fill(str(response["input"]), 80))
print()
print(textwrap.fill(str(response["output"]), 80))
print()
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
for msg in msg_history:
  print(msg)

# CLI for Agent (Chatbot with memory)

In [10]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
messages = [{"role": "assistant", "content": "How can I help you?"}]

In [12]:
#prompt = "Give me the premium for an insured sum of 10000000"
prompt = "The industry is chemical production"
#prompt = "And the temperature in Celsius?"
messages.append({"role": "user", "content": prompt})
print("user: " + prompt)
response = agent_executor.invoke({"input": prompt})
msg = {"role": "assistant", "content": response["output"]}
messages.append(msg)
print(textwrap.fill("assistant: " + response["output"], 80))
print("\nHistory:")
for msg in messages:
    #st.chat_message(msg["role"]).write(msg["content"])
    print(textwrap.fill(msg["role"] + ": " + msg["content"], 80))

user: The industry is chemical production


> Entering new AgentExecutor chain...

Invoking: `calculate_insurance_premium` with `{'insured_sum': 10000000, 'industry': 'chemical production'}`


2500000.0The premium for an insured sum of 10,000,000 in the chemical production industry is 2,500,000.

> Finished chain.
assistant: The premium for an insured sum of 10,000,000 in the chemical
production industry is 2,500,000.

History:
assistant: How can I help you?
user: Give me the premium for an insured sum of 10000000
assistant: Sure, I can help with that. However, I also need to know the industry
of the customer to calculate the insurance premium. Could you please provide
that information?
user: The industry is chemical production
assistant: The premium for an insured sum of 10,000,000 in the chemical
production industry is 2,500,000.


In [ ]:
print("Memory from response object:")
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
for msg in msg_history:
  print(msg)

# Work in progress...

In [ ]:
os.environ["LANGCHAIN_TRACING"] = "true"
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')

In [ ]:
# Tool 1
def multiplier(a: float, b: float) -> float:
    """Multiply the provided floats."""
    return a * b

tool = StructuredTool.from_function(multiplier)

agent_executor = initialize_agent(
    [tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
prompt = "What is 3 times 4"
response_text = agent_executor.run(prompt)
print(response_text)
response = agent_executor.invoke({"input": prompt})
#print(textwrap.fill(str(response["output"]), 80))


In [ ]:
# Tool 2
def calculate_insurance_premium(insured_sum: int, industry: str) -> float:
    """Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer."""
    premium = insured_sum*0.25
    return premium

tool = StructuredTool.from_function(calculate_insurance_premium, return_direct=False)

agent_executor = initialize_agent(
    [tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

prompt = "Give me the premium for an insured sum of 10000000"
prompt = "Kalkuliere die Versicherungsprämie für eine Versicherungssumme von 10000000, die Branche ist Metallbau"
response_text = agent_executor.run(prompt)
print(response_text)
response = agent_executor.invoke({"input": prompt})
#print(textwrap.fill(str(response["output"]), 80))
